In [1]:
!pip install transformers datasets evaluate wandb torchmetrics torchinfo ftfy regex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-mlpqt0j7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-mlpqt0j7
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=2ea3cf0b2e0064f0be681bc825a6a48dcff1490dd0491c9a4e39639253df16af
  Stored in directory: /tmp/pip-ephem-wheel-cache-cnbodan0/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [3]:
from huggingface_hub import login
from google.colab import userdata
import wandb

login(token=userdata.get('HF_TOKEN'))
wandb.login(key=userdata.get('WANDB_API_KEY'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast
from transformers import AutoModelForUniversalSegmentation, AutoProcessor
from torchmetrics.classification import Accuracy, MulticlassAccuracy, MulticlassJaccardIndex, JaccardIndex
import albumentations as A
from tqdm.auto import tqdm
from huggingface_hub import notebook_login
from datasets import load_dataset
import wandb
import evaluate
import statistics
from copy import deepcopy
from huggingface_hub import hf_hub_download
import torchinfo

import numpy as np
import pandas as pd
import random
import requests
import json
from pathlib import Path
import os
from typing import List, Dict, Tuple
import warnings
from PIL import Image as PILImage
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
#warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True

In [5]:
model_ckpt = f"shi-labs/oneformer_cityscapes_swin_large"
test_processor = AutoProcessor.from_pretrained(model_ckpt)
test_processor.image_processor

preprocessor_config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

cityscapes_panoptic.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/812 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

OneFormerImageProcessor {
  "_max_size": 2048,
  "class_info_file": "cityscapes_panoptic.json",
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": true,
  "ignore_index": 255,
  "image_mean": [
    0.48500001430511475,
    0.4560000002384186,
    0.4059999883174896
  ],
  "image_processor_type": "OneFormerImageProcessor",
  "image_std": [
    0.2290000021457672,
    0.2239999920129776,
    0.22499999403953552
  ],
  "metadata": {
    "0": "road",
    "1": "building",
    "10": "sky",
    "11": "person",
    "12": "rider",
    "13": "car",
    "14": "truck",
    "15": "bus",
    "16": "train",
    "17": "motorcycle",
    "18": "bicycle",
    "2": "sidewalk",
    "3": "wall",
    "4": "fence",
    "5": "pole",
    "6": "traffic light",
    "7": "traffic sign",
    "8": "vegetation",
    "9": "terrain",
    "class_names": [
      "road",
      "building",
      "sidewalk",
      "wall",
      "fence",
      "pole",
      "traffic light",
      "traff

In [6]:
model_ckpt_mask2former = f"facebook/mask2former-swin-large-cityscapes-panoptic"
test_model_mask2former = AutoModelForUniversalSegmentation.from_pretrained(model_ckpt_mask2former, ignore_mismatched_sizes=True)
test_model_mask2former

config.json:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/866M [00:00<?, ?B/s]

Mask2FormerForUniversalSegmentation(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=192, out_features=192, bias=True)
                      (key): Linear(in_features=192, out_features=192, bias=True)
                     

In [7]:
model_ckpt = f"shi-labs/oneformer_cityscapes_swin_large"
test_model = AutoModelForUniversalSegmentation.from_pretrained(
    model_ckpt, is_training=True, ignore_mismatched_sizes=True
)

def print_trainable_parameters(model, print_msg):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for name, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"\n-----{print_msg}......")
    print(
        f"trainable params: {trainable_params / 1e+6:.2f}M || all params: {all_param / 1e+6:.2f}M || trainable%: {100 * trainable_params / all_param:.2f}%"
    )

print_trainable_parameters(test_model, print_msg="Base Model Trainable Params")
# Freeze the model backbone params
for name, params in test_model.named_parameters():
    if name.startswith("model.pixel_level_module"): #195.20M'
            mdl_keywords = ["decoder.mask_projection", "decoder.adapter_1", "decoder.layer_1"] #35.33 to 36.03M when added
            if not any(keyword in name for keyword in mdl_keywords):
                params.requires_grad = False
# Get the last layer of the decoder encoder layers and Set require_grad to True
last_layer_params = test_model.model.pixel_level_module.decoder.encoder.layers[-1].parameters() #36.77M
for param in last_layer_params:
    param.requires_grad = True

print_trainable_parameters(test_model, print_msg="Frozen Model Trainable Params")

#test_model.model.pixel_level_module.decoder.encoder.layers[-1], test_model.model.pixel_level_module.decoder.mask_projection, test_model.model.pixel_level_module.decoder.adapter_1, test_model.model.pixel_level_module.decoder.layer_1,

config.json:   0%|          | 0.00/77.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/879M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of OneFormerForUniversalSegmentation were not initialized from the model checkpoint at shi-labs/oneformer_cityscapes_swin_large and are newly initialized: ['model.text_mapper.prompt_ctx.weight', 'model.text_mapper.text_encoder.ln_final.bias', 'model.text_mapper.text_encoder.ln_final.weight', 'model.text_mapper.text_encoder.positional_embedding', 'model.text_mapper.text_encoder.token_embedding.weight', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.bias', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.weight', 'model.text_mapper.text_encoder.transformer.


-----Base Model Trainable Params......
trainable params: 236.32M || all params: 236.32M || trainable%: 100.00%

-----Frozen Model Trainable Params......
trainable params: 36.77M || all params: 236.32M || trainable%: 15.56%


In [8]:
params = []
for name, param in test_model.named_parameters():
        print(name, param.shape)
        params.append(param)

len(params)

model.pixel_level_module.encoder.embeddings.patch_embeddings.projection.weight torch.Size([192, 3, 4, 4])
model.pixel_level_module.encoder.embeddings.patch_embeddings.projection.bias torch.Size([192])
model.pixel_level_module.encoder.embeddings.norm.weight torch.Size([192])
model.pixel_level_module.encoder.embeddings.norm.bias torch.Size([192])
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.layernorm_before.weight torch.Size([192])
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.layernorm_before.bias torch.Size([192])
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.relative_position_bias_table torch.Size([529, 6])
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.query.weight torch.Size([192, 192])
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.query.bias torch.Size([192])
model.pixel_level_module.encoder.encoder.layers.0.blocks.0.attention.self.key.weight torch.Size([192, 192])
model.pi

882

In [9]:
for name, param in test_model.named_parameters():
    if "text_mapper" in name:
        print(name, param.shape)

model.text_mapper.text_encoder.positional_embedding torch.Size([77, 256])
model.text_mapper.text_encoder.transformer.layers.0.self_attn.in_proj_weight torch.Size([768, 256])
model.text_mapper.text_encoder.transformer.layers.0.self_attn.in_proj_bias torch.Size([768])
model.text_mapper.text_encoder.transformer.layers.0.self_attn.out_proj.weight torch.Size([256, 256])
model.text_mapper.text_encoder.transformer.layers.0.self_attn.out_proj.bias torch.Size([256])
model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.weight torch.Size([256])
model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.bias torch.Size([256])
model.text_mapper.text_encoder.transformer.layers.0.mlp.fc1.weight torch.Size([1024, 256])
model.text_mapper.text_encoder.transformer.layers.0.mlp.fc1.bias torch.Size([1024])
model.text_mapper.text_encoder.transformer.layers.0.mlp.fc2.weight torch.Size([256, 1024])
model.text_mapper.text_encoder.transformer.layers.0.mlp.fc2.bias torch.Size([256])
model.text_ma

In [10]:
test_model.model.pixel_level_module.decoder.encoder.layers[-1], test_model.model.pixel_level_module.decoder.mask_projection, test_model.model.pixel_level_module.decoder.adapter_1, test_model.model.pixel_level_module.decoder.layer_1, test_model.model.pixel_level_module.decoder

(OneFormerPixelDecoderEncoderLayer(
   (self_attn): OneFormerPixelDecoderEncoderMultiscaleDeformableAttention(
     (sampling_offsets): Linear(in_features=256, out_features=192, bias=True)
     (attention_weights): Linear(in_features=256, out_features=96, bias=True)
     (value_proj): Linear(in_features=256, out_features=256, bias=True)
     (output_proj): Linear(in_features=256, out_features=256, bias=True)
   )
   (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
   (fc1): Linear(in_features=256, out_features=1024, bias=True)
   (fc2): Linear(in_features=1024, out_features=256, bias=True)
   (final_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
 ),
 Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1)),
 Sequential(
   (0): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
   (1): GroupNorm(32, 256, eps=1e-05, affine=True)
 ),
 Sequential(
   (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False

In [11]:
def prompt_params(model):
    params = []
    for name, param in model.named_parameters():
        if "text_mapper" in name:
            params.append(param)
    return params

textmapper_params = prompt_params(test_model)
prompt_group = {'params': textmapper_params, "lr": 1e-2}
print('num of params in prompt textmapper:', len(textmapper_params))
adam_parameters = [prompt_group]

def pretrained_model_params(model):
    params = []
    for name, param in model.named_parameters():
        if "text_mapper" not in name:
            params.append(param)
    return params


pretrained_parameters = pretrained_model_params(test_model)
print('num of params in pretrained modules: ', len(pretrained_parameters))
base_group = {'params': pretrained_parameters, 'lr': 1e-4}
adam_parameters.append(base_group)

# optimizer = torch.optim.AdamW(params=adam_parameters, lr=1e-3, weight_decay=0.1)



num of params in prompt textmapper: 81
num of params in pretrained modules:  801


In [12]:
from copy import deepcopy

def get_labels():
    data_directory = Path(".")
    json_file_path = data_directory / "labels_info.json"
    if json_file_path.is_file():
        print(f"[INFO]: Found {json_file_path}.Skipping Download...")
    else:
        print("[INFO]: Downloading labels_info.json from hub")
        json_file_path = hf_hub_download(
            repo_id="BhavanaMalla/railsem19-semantic-expanded",
            filename="labels_info.json",
            repo_type="dataset",
            local_dir=data_directory
        )
    with open(json_file_path, "r") as f:
        labels_info = json.load(f)
    id2label = labels_info["id2label"]
    label2id = labels_info["label2id"]
    labels = labels_info["labels"]
    color_palette = labels_info["color_palette"]

    #add the background label
    id2label["19"] = "background"
    label2id["background"] = 19
    labels.append("background")
    color_palette.append([0, 0, 0])

    # correcting the labels
    id2label = {int(key): value.replace('-', '_') for key, value in id2label.items()}
    label2id = {key.replace('-', '_'): value for key, value in label2id.items()}
    labels = {label.replace('-', '_') for label in labels}
    return {"id2label": id2label, "label2id": label2id, "labels": labels, "color_palette": color_palette}

def load_railsem_dataset(device=None):
    print(f"[INFO]: Extracting Railsem19 dataset from hub on device {device}")
    railsem_ds = load_dataset("BhavanaMalla/railsem19-semantic-expanded")
    return railsem_ds

In [13]:
import torchvision.transforms as tvt

def get_transforms():
    # Transforms
    image_transforms = tvt.Compose([tvt.ToTensor(),]) #ToTensor scales between 0 and 1, if we use normalize, the values goes beyond 0 and 1, the range could be outside of [0, 1] and would ideally have a zero mean and unit variance.
    mask_transforms = tvt.Compose([tvt.PILToTensor(),
                                  RemapBackground(),
                                  RemapLabels()]) #Converts a PIL Image (H x W x C) to a Tensor of shape (C x H x W). doesnot scale hence using this for masks

    return image_transforms, mask_transforms

class RemapBackground():
    """ Remap background to label 19 """
    def __call__(self, mask):
        return torch.where(mask > 18, 19, mask)

class RemapLabels():
    def __init__(self):
        self.class_mapping = get_labels()["id2label"]
        print("class_mapping")
        print(self.class_mapping)
        self.class_coding = {v: k for k, v in self.class_mapping.items()}
        print("class_coding")
        print(self.class_coding)
        # MODIFICATIONS TO ORIGINAL CLASSES
        self.modified_labels = {
            0:["road"], 1:["sidewalk"], 2:['construction', 'fence'], 3:['rail_raised','rail_embedded'], 4:['pole','traffic_light','traffic_sign'],
            5:['sky'], 6:['human'], 7:['tram_track', 'rail_track'], 8:['car','truck'], 9:['on_rails'], 10:['vegetation'], 11:['trackbed'], 12:['background','terrain']
        }
        self.modified_ids = {}
        for k, v in self.modified_labels.items():
            self.modified_ids[k] = [self.class_coding[label] for label in v]
        print("modified_ids")
        print(self.modified_ids)

    def __call__(self, mask):
        final_label = np.zeros_like(mask)
        for k, v in self.modified_ids.items():
            specific_label = np.zeros_like(mask)
            specific_label = np.where(np.isin(mask, np.array(v)), 1, 0)
            specific_label *= k
            final_label = np.add(final_label, specific_label)
        return torch.from_numpy(final_label)

In [14]:
class CustomDataset(Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"] #.transpose(2, 0, 1) #(1080, 1920, 3) HXWxC
        print(f"original_img shape {type(image)}")
        semantic_mask = self.dataset[idx]["semantic_mask_label"] #((1080, 1920)) HXW
        print(f"semantic mask shape {type(semantic_mask)}")
        # original_image = deepcopy(image)
        # target_mask = deepcopy(semantic_mask)

        if self.transforms:
            image_transforms, target_transforms = self.transforms
            # random.seed(self.seed_value)
            # torch.manual_seed(self.seed_value)
            transformed_image = image_transforms(image)
            transformed_mask = target_transforms(semantic_mask)
            print("unique labels")
            my_transform = tvt.Compose([tvt.PILToTensor(),
                                        RemapBackground()])
            print(torch.unique(my_transform(semantic_mask)))
            print(torch.unique(transformed_mask))
            # transformed = self.transforms(image=image, mask=semantic_mask)
            # transformed_image, transformed_mask = transformed["image"], transformed["mask"]
            # # Transpose the transformed image to C x H x W
            # transformed_image = transformed_image.transpose(2, 0, 1)
        # else:
        #     transformed_image, transformed_mask = image.transpose(2, 0, 1), semantic_mask
        #print(transformed_image.shape, transformed_mask.shape, original_image.shape, target_mask.shape) #(3, 512, 512) (512, 512) (1080, 1920, 3) (1080, 1920)
        else:
            image_transforms, mask_transforms = get_transforms()
            transformed_image = image_transforms(image)
            transformed_mask = mask_transforms(semantic_mask)
        # return  transformed_image, transformed_mask, original_image, target_mask
        print(f"transformed_image shape {transformed_image.shape}")
        return  {"image": transformed_image, "mask": transformed_mask}

In [15]:
def print_trainable_parameters(model, print_msg):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for name, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"\n-----{print_msg}......")
    print(
        f"trainable params: {trainable_params / 1e+6:.2f}M || all params: {all_param / 1e+6:.2f}M || trainable%: {100 * trainable_params / all_param:.2f}%"
    )

def get_model_and_processor(ckpt_dataset_name, ckpt_name,
                            id2label=None, label2id=None, labels=None,
                            is_train=False):
    id2label = {str(k): v for k, v in id2label.items()}
    print("[INFO]: Prepare the Model and the Processor...")
    model_ckpt = f"shi-labs/oneformer_{ckpt_dataset_name}_{ckpt_name}_large"
    if is_train:
        print("[INFO]: Model is in training mode...")

        model = AutoModelForUniversalSegmentation.from_pretrained(
            model_ckpt, is_training=is_train,
            id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
        )
    # Processor
    processor = AutoProcessor.from_pretrained(
        model_ckpt,
        ignore_index=255,
        do_reduce_labels=False,
        do_normalize=True,
        do_rescale=True,
        do_resize=False
    )
    metadata = deepcopy(id2label)
    metadata["thing_ids"] = [11, 12, 13, 14]
    metadata["class_names"] = labels
    processor.image_processor.metadata = metadata
    processor.image_processor.num_labels = len(id2label)
    processor.image_processor.num_text = model.config.num_queries - model.config.text_encoder_n_ctx

    # Parameter Freezing
    if is_train:
        # Freeze the model backbone params
        for name, params in model.named_parameters():
            if name.startswith("model.pixel_level_module"):
                params.requires_grad = False
        print_trainable_parameters(
            model, print_msg="Frozen Model Trainable Params"
        )
    return model, processor

In [16]:
import torch
import torchvision.transforms as tvt
from torch.utils.data import DataLoader, Dataset, Subset
import torchvision
from torchvision.transforms.functional import crop
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F


class PatchQueue(Dataset):
    '''
    Extract random patches from an image -> put them in queue -> feed them to the dataloader

    ref : https://github.com/fepegar/torchio/blob/main/src/torchio/data/queue.py
    '''
    def __init__(
            self, dataset, max_length, # Queue Length
            samples_per_image, # Patches per image
            queue_workers = 2, # Num Workers
            patch_size = [512, 512], # Patch Size
            shuffle = False
        ):

        self.dataset = dataset
        self.max_length = max_length
        self.queue_workers = queue_workers
        self.samples_per_image = samples_per_image
        self.shuffle_queue = shuffle
        self.patch_size = patch_size
        self._images_iterable = None
        self.patch_list = []
        self._num_sampled_images = 0
        self.steps_per_epoch = len(self.dataset) * self.samples_per_image
        self.resize_transform = tvt.Resize(patch_size[::-1])  # Reverse patch_size for Resize

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, item):
        if not self.patch_list:
            print("Patch List is empty.")
            self._fill()
            self.patch_list.reverse()
        sample_patch = self.patch_list.pop()
        return sample_patch

    @staticmethod
    def _get_first_item(batch):
        if isinstance(batch[0], dict):
            print(batch[0].keys())
        else:
            print(batch[0])
        return batch[0]

    def initialize_images_iterable(self):
        self._images_iterable = self._get_images_iterable()

    @property
    def images_iterable(self):
        if self._images_iterable is None:
            self.initialize_images_iterable()
        return self.images_iterable

    def _get_images_iterable(self):
        print("dataloader datset\n")
        loader = DataLoader(self.dataset,
                            num_workers = self.queue_workers,
                            batch_size=1,
                            collate_fn=self._get_first_item,
                            shuffle = self.shuffle_queue
                            ) #make it ddp compatible
        self._num_sampled_images = 0
        return iter(loader)

    def _get_next_image(self):
        try:
            image = next(self._images_iterable)
        except Exception as e:
            print(f"Excepton: {e}")
            self.initialize_images_iterable()
            image = next(self._images_iterable)
        return image

    def extract_patches(self, image_pair, samples_per_image, patch_size):
        image_patches = []
        img_height, img_width = image_pair['image'].shape[-2], image_pair['image'].shape[-1]
        # Check if image size is smaller than patch size
        if img_height < patch_size[0] or img_width < patch_size[1]:
            # Resize the image to match the patch size
            image_pair['image'] = self.resize_transform(image_pair['image'])
            image_pair['mask'] = self.resize_transform(image_pair['mask'])
            img_height, img_width = image_pair['image'].shape[-2], image_pair['image'].shape[-1]
        for i in range(samples_per_image):
            left = torch.randint(low=0, high=img_width-patch_size[0], size=[1,]).item()
            top = torch.randint(low=0, high=img_height-patch_size[1], size=[1,]).item()
            cropped_image = crop(img=image_pair['image'], top=top, left=left, height=patch_size[1], width=patch_size[0])
            cropped_labels = crop(img=image_pair['mask'], top=top, left=left, height=patch_size[1], width=patch_size[0])
            image_patches.append({'image':cropped_image, 'mask':cropped_labels})
        return image_patches


    def _fill(self):
        while True:
            image_pair = self._get_next_image()
            samples_per_image = self.samples_per_image
            patch_size = self.patch_size
            print("Extract patches")
            patches = self.extract_patches(image_pair, samples_per_image,patch_size)
            self.patch_list.extend(patches)
            self._num_sampled_images += 1
            islistfull = len(self.patch_list) >= self.max_length
            print("end")
            print(len(self.patch_list))
            for i, patch in enumerate(self.patch_list):
                print(i, patch.keys(), patch["image"].shape, patch["mask"].shape)
            if islistfull:
                break




# def show(imgs):
#     fig, ax = plt.subplots(nrows = 3,ncols = 3, squeeze = False)
#     for i, (img, label, im_full) in enumerate(zip(imgs['image'],imgs['label'], imgs['full_image'])):
#         image = F.to_pil_image(img)
#         im_full = F.to_pil_image(im_full)
#         label = label.squeeze()
#         ax[0,0].imshow(image)
#         ax[1,0].imshow(label,cmap = "tab20")
#         ax[2,0].imshow(im_full)
#     plt.show()



# if __name__ == '__main__':
#     train_transform_img = tvt.Compose([

#         tvt.ToTensor(),
#     ])

#     train_transform_target = tvt.Compose([

#         tvt.PILToTensor(),
#     ])

#     from dataset_rail import RailSem19Dataset

#     # dataset = torchvision.datasets.Cityscapes(root="DATA/CITY_SCAPES", split="train",mode="fine",
#     #                                             target_type="semantic",transform=train_transform_img, target_transform=train_transform_target)
#     ds = RailSem19Dataset(image_transform=train_transform_img)

#     qob = PatchQueue(dataset = ds, max_length=4, samples_per_image = 5, patch_size = [1000,500])
#     dl = DataLoader(qob,1)
#     for i,data in enumerate(dl):
#         show(data)
#         if i >= 20:
#             break

In [17]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

def show(imgs):
    fig, ax = plt.subplots(nrows = 3,ncols = 3, squeeze = False)
    # imgs['full_image']
    for i, (img, label) in enumerate(zip(imgs['original_images'],imgs['original_segmentation_maps'])):
        image = F.to_pil_image(img)
        # im_full = F.to_pil_image(im_full)
        label = label.squeeze()
        ax[0,0].imshow(image)
        ax[1,0].imshow(label, cmap = "tab20")
        # ax[2,0].imshow(im_full)
    plt.show()

In [36]:
from torchvision.transforms.functional import to_pil_image

def log_selected_indices_data(selected_indices_data, id2label):
    original_images, targets, pred_segmaps = zip(*selected_indices_data)
    # Convert to numpy arrays
    print("log_selected_indices_data before cat")
    print(len(original_images), original_images[0].shape)
    print(len(targets), targets[0].shape)
    print(len(pred_segmaps), pred_segmaps[0].shape)
    original_images = torch.stack(original_images).detach().cpu().numpy()
    targets = torch.stack(targets).detach().cpu().numpy()
    pred_segmaps = torch.stack(pred_segmaps).detach().cpu().numpy()
    print("log_selected_indices_data")
    print(original_images.shape)
    print(targets.shape)
    print(pred_segmaps.shape)
    # Log the data using wandb_mask_log
    table_data = wandb_mask_log(
        original_images, targets, pred_segmaps, id2label
    )
    return table_data

# util function for generating interactive image mask from components
def wandb_mask_log(images, true_seg_masks, pred_seg_masks, id2label):
    new_id2label = {int(key): value for key, value in id2label.items()}
    print(f"wandbmasklog id2label {new_id2label}")
    table_data = []
    for img, gt_mask, pred_mask in zip(images, true_seg_masks, pred_seg_masks):
        print(np.unique(gt_mask), np.unique(pred_mask))
        try:
            img *= 255
            img = img.transpose((1, 2, 0))
            img = (img).astype(np.uint8)
            gt_mask = (gt_mask).astype(np.uint8)
            pred_mask = (pred_mask).astype(np.uint8)
            pil_image = to_pil_image(img)
            table_data.append(wandb.Image(
                pil_image, masks={
                    "ground truth": {
                        "mask_data": gt_mask,
                        "class_labels": new_id2label
                    },
                    "prediction": {
                        "mask_data": pred_mask,
                        "class_labels": new_id2label
                    }
                }
            ))
        except Exception as e:
            # Handle the exception related to PILImage broken chunk ocassionally
            print(f"Exception in creating wandb.Image: {e}")
            # Skip this iteration and move to the next image
            continue
    return table_data

In [40]:
from torchmetrics.classification import Accuracy, MulticlassAccuracy, MulticlassJaccardIndex, JaccardIndex

class ModelResults:
    def __init__(self):
        pass

class OneformerTrainer:
    def __init__(self, args, run, gpu_id):
        self.seed = args.seed
        self.railsem_ds = load_railsem_dataset(gpu_id)
        labels_info = get_labels()
        self.id2label = labels_info["id2label"]
        self.label2id = labels_info["label2id"]
        self.labels = labels_info["labels"]
        self.color_palette = labels_info["color_palette"]

        print(f"[INFO]: id2label: {self.id2label}")
        print(f"[INFO]: Setting the seed {self.seed}")
        set_seeds(self.seed)
        self.model, self.processor = get_model_and_processor(
            args.ckpt_dataset_name, args.ckpt_model_name, id2label=self.id2label, label2id=self.label2id,
            labels=self.labels, is_train=args.is_train
        )
        self.is_distributed = False
        self.epochs = args.num_epochs
        self.batch_size = args.batch_size
        self.learning_rate = args.lr
        self.device = gpu_id
        self.model.to(self.device)
        self.is_train = args.is_train
        if self.is_train:
            params_groups = self.get_params_groups() #create param groups
        else:
            params_groups = self.model.parameters()
        self.optimizer = AdamW(params=params_groups, lr=self.learning_rate)
        self.scheduler = ReduceLROnPlateau(self.optimizer, "min", factor=0.5, patience=5, threshold=0.0001, min_lr=1e-6)
        self.scaler = GradScaler()
        self.best_loss = 1e+5
        self.start_epoch = 0
        self.global_step = 0
        if run is not None:
            print(f"[INFO]: Run object: {run}")
            run.config["classes"] = len(self.id2label)
            self.wandb_config = run.config
            self.run = run
        self.log = self.run is not None
        # Metrics and results
        self.metric_names = ["loss", "mean_iou", "mean_accuracy", "overall_accuracy", "per_category_iou", "per_category_accuracy"]
        self.metric_types = ["train", "val"]
        self.model_results = ModelResults()
        #calculates metrics(iou, acc) for each label and applies no reduction, we ignore background class
        self.iou_per_cls = JaccardIndex(task="multiclass", num_classes=len(self.id2label), average="none", ignore_index=255).to("cpu")
        self.acc_per_cls = Accuracy(task="multiclass", num_classes=len(self.id2label), average="none", ignore_index=255).to("cpu")
        # Calculate metrics(miou, Overallacc) for each class separately and average the metrics across classes after weighing each class by its support = TP + FN
        self.IoU_weighted = JaccardIndex(task="multiclass", num_classes=len(self.id2label), average='weighted', ignore_index=255).to("cpu")
        self.acc_overall_weighted = Accuracy(task="multiclass", num_classes=len(self.id2label), average='weighted', ignore_index=255).to("cpu")
        self.jaccard_index = JaccardIndex(task="multiclass", num_classes=len(self.id2label), average='weighted', ignore_index=255).to("cpu")
        # Val metrics
        self.val_iou_per_cls = JaccardIndex(
            task="multiclass", num_classes=len(self.id2label),
            average="none", ignore_index=255).to(self.device)
        self.val_acc_per_cls = Accuracy(
            task="multiclass", num_classes=len(self.id2label),
            average="none", ignore_index=255).to(self.device)
        # Calculate metrics(miou, Overallacc) for each class separately and average the metrics across classes after weighing each class by its support = TP + FN
        self.val_IoU_weighted = JaccardIndex(
            task="multiclass", num_classes=len(self.id2label),
            average='weighted', ignore_index=255).to(self.device)
        self.val_acc_overall_weighted = Accuracy(
            task="multiclass", num_classes=len(self.id2label),
            average="weighted", ignore_index=255).to(self.device)
        self.val_jaccard_index = JaccardIndex(
            task="multiclass", num_classes=len(self.id2label),
            average="weighted", ignore_index=255).to(self.device)

    def _get_datasets(self):
        print("[INFO]: Preparing the Datasets...")
        try:
            if "data" in self.railsem_ds:
                print("[INFO]: No splits found.. Generating splits")
                print(f"[INFO]: Total images: {len(self.railsem_ds['data'])}")
                splits = self.railsem_ds["data"].train_test_split(
                    test_size=0.05, shuffle=True
                )
                random.seed(1)
                train_split = splits["train"].select(indices=(range(5)))
                val_split = splits["test"].select(indices=(range(3)))
                print(f"[INFO]: Total Training images: {len(train_split)}")
                print(f"[INFO]: Total Validation images: {len(val_split)}")
            else:
                raise ValueError("Data splits not in required format")
        except ValueError as e:
            print(e)
        # Get the transforms
        transforms = get_transforms()
        # Custom Dataset
        train_dataset = CustomDataset(train_split, transforms)
        val_dataset = CustomDataset(val_split, transforms)
        print("patch queue calling")
        train_queue = PatchQueue(dataset=train_dataset, max_length=10, samples_per_image=5, patch_size=[512, 512])#[1024, 1024]
        val_queue = PatchQueue(dataset=val_dataset, max_length=10,samples_per_image=5, patch_size=[512, 512])
        print(type(train_queue))
        return train_dataset, val_dataset, train_queue, val_queue

    def get_params_groups(self):
        params_group = []
        textmapper_params = []
        pretrained_params = []
        for name, param in self.model.named_parameters():
            if "text_mapper" in name:
                textmapper_params.append(param)
            else:
                pretrained_params.append(param)
        params_group.append({"params": textmapper_params, "lr": 1e-2})
        params_group.append({"params": pretrained_params})  # "lr": 1e-4
        print("[INFO]: Num of params in prompt textmapper group:", len(textmapper_params))
        print("[INFO]: Num of params in pretrained group:", len(pretrained_params))
        return params_group

    def _prepare_dataloader(self, dataset: torch.utils.data.Dataset,
                           batch_size: int, is_train):
        print("inside")
        sampler = None
        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            pin_memory=True,
            shuffle=(is_train and sampler is None),
            sampler=sampler,
            collate_fn=self._collate_fn,
        )
        return dataloader

    def _collate_fn(self, batch):
        print("my collate")
        images = torch.stack([sample['image'] for sample in batch])
        segmentation_maps = torch.cat([sample['mask'] for sample in batch])
        print("collate")
        print(images.shape)
        print(segmentation_maps.shape)
        batch = self.processor(images,
                               segmentation_maps=segmentation_maps,
                               task_inputs=["semantic"] * len(images),
                               return_tensors="pt")
        batch["original_images"] = images
        batch["original_segmentation_maps"] = segmentation_maps
        return batch

    def log_category_chart(self, metric_values, title):
        # Ensure metric_values is a list of tensors to handle both single last epoch and all epochs metrics
        if not isinstance(metric_values, list):
            metric_values = [metric_values]

        # Converting nan to zeros
        metric_values = [torch.where(torch.isnan(tensor), torch.tensor(0.0), tensor) for tensor in metric_values]

        # Stack if it's a list of tensors
        avg_metric_values = torch.stack(metric_values).nanmean(dim=0) if len(metric_values) > 1 else metric_values[0]

        data = [[label, acc] for (label, acc) in zip(
            self.processor.image_processor.metadata["class_names"],
            avg_metric_values)
        ]
        table = wandb.Table(data=data, columns=["Class Name", title])
        self.run.log({
            f"{title}": wandb.plot.bar(
                table, "Class Name", title,
                title=f"{title}")
        })

    def log_metrics(self, epoch, global_step, metrics_to_log=None):
        for metric_type in self.metric_types:
            print(f"[INFO]: Logging {metric_type} metrics")
            metric_values = {}
            # If metrics_to_log is not specified (or is None), it defaults to logging all metrics
            for metric_name in metrics_to_log or self.metric_names:
                full_metric_name = f"{metric_type}_{metric_name}"
                metric_values[full_metric_name] = getattr(self.model_results, full_metric_name)[-1] # get the last one in the respective metric list
            # Log all metrics for the specified metric type in a single step
            metric_values["custom_step"] = int(global_step)
            self.run.log(metric_values)

    def get_all_metrics(self):
        all_metrics = {}
        for metric_type in self.metric_types:
            for metric_name in self.metric_names:
                full_metric_name = f"{metric_type}_{metric_name}"
                all_metrics[full_metric_name] = getattr(self.model_results, full_metric_name)
        return all_metrics

    def train_step(self, train_dataloader: torch.utils.data.DataLoader):
        running_train_loss = 0.0
        num_samples = 0
        total_steps = len(train_dataloader) * self.epochs

        running_iou =  []
        self.model.train()
        for step, batch in enumerate(tqdm(train_dataloader)):
            print("batch")
            print(batch.keys())
            # for k, v in batch.items():
            #     if isinstance(v, torch.Tensor):
            #         print(k, v.shape)
            #     else:
            #         print(k, len(v))
            self.optimizer.zero_grad()
            batch_dict = {
                "pixel_values": batch["pixel_values"].to(self.device),
                "mask_labels": [labels.to(self.device)
                                for labels in batch["mask_labels"]],
                "class_labels": [labels.to(self.device)
                                 for labels in batch["class_labels"]],
                "pixel_mask": batch["pixel_mask"].to(self.device),
                "text_inputs": batch["text_inputs"].to(self.device),
                "task_inputs": batch["task_inputs"].to(self.device),
            }
            # Forward Pass
            with autocast():
                outputs = self.model(**batch_dict)
                train_loss = outputs.loss
            # print("outputs")
            # for k, v in outputs.items():
            #     if isinstance(v, torch.Tensor):
            #         print(k, v.dtype)
            #     else:
            #         print(k, len(v))
            # print(outputs["class_queries_logits"].float().dtype)
            # print(outputs["masks_queries_logits"].float().dtype)
            # outputs["class_queries_logits"] = outputs["class_queries_logits"].float()
            # outputs["masks_queries_logits"] = outputs["masks_queries_logits"].float()

            running_train_loss += train_loss.long().item()
            train_batch_size = batch["pixel_values"].size(0)
            num_samples += train_batch_size
            print(f"Train Loss after {str(num_samples).zfill(6)} examples:"
                  f"{train_loss.item():.3f}\n")

            # Post process segmentation
            original_images = batch["original_images"]
            print("original images")
            for img in original_images:
                print(img.shape) #uint8
            target_sizes = [(img.shape[-2], img.shape[-1]) for img in original_images]
            predicted_segmentation_maps = self.processor.post_process_semantic_segmentation(outputs, target_sizes=target_sizes)
            print(outputs.keys())
            print(outputs["masks_queries_logits"].shape), print(outputs["class_queries_logits"].shape)
            print("predicted_segmentation_maps")
            print(predicted_segmentation_maps[0].shape, predicted_segmentation_maps[1].shape)
            # Get ground truth segmentation maps and move both target and preds to cpu
            # target = torch.stack([torch.from_numpy(arr) for arr in batch["original_segmentation_maps"]]).to("cpu")
            #`preds` with shape=torch.Size([2, 3, 512]) and `target` with shape=torch.Size([2, 512, 512])
            # target = torch.stack([g for g in batch["original_segmentation_maps"]]).to("cpu")
            target = batch["original_segmentation_maps"].to("cpu")
            preds = torch.stack([t.detach().cpu() for t in predicted_segmentation_maps])
            print("tensor dtypes")
            print(target.dtype, target.shape) #torch.uint8
            print(preds.dtype, preds.shape) #torch.int64
            print(target.long().dtype) #torch.int64
            print(preds.long().dtype) #torch.int64

            # Metrics calculation per batch
            iou_per_cls_batch = self.iou_per_cls(preds.long(), target.long())
            acc_per_cls_batch = self.acc_per_cls(preds.long(), target.long())
            IoU_batch = self.IoU_weighted(preds.long(), target.long())
            acc_overall_batch = self.acc_overall_weighted(preds.long(), target.long())

            #Jackard metric - can be deleted later it is same as IoU_weighted
            running_iou.append(self.jaccard_index(preds.long(), target.long()).item())

            # Log loss to wandb every 10 batches i.e after every 10 steps
            self.global_step += 1
            if ((step + 1) % 10) == 0 or step == len(train_dataloader) - 1:
                if self.log:
                    self.run.log({
                        "custom_step": int(self.global_step),
                        "train_step_loss": train_loss.item()
                    })
            # Backward pass
            self.scaler.scale(train_loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
            del batch, outputs

        # after training the epoch
        train_epoch_loss = running_train_loss / len(train_dataloader)
        iou_per_cls_epoch = self.iou_per_cls.compute()
        mIoU_epoch = self.IoU_weighted.compute()
        jacc_miou_train = statistics.mean(running_iou)
        acc_per_cls_epoch = self.acc_per_cls.compute()
        acc_overall_epoch = self.acc_overall_weighted.compute()
        mean_acc_epoch = torch.nanmean(acc_per_cls_epoch)
        train_metrics_epoch = {'mean_iou': mIoU_epoch.item(),
                       'mean_accuracy': mean_acc_epoch.item(),
                       'overall_accuracy': acc_overall_epoch.item(),
                       'per_category_iou': iou_per_cls_epoch,
                       'per_category_accuracy': acc_per_cls_epoch}
        # Reset the metric states after the epoch
        self.iou_per_cls.reset()
        self.IoU_weighted.reset()
        self.acc_per_cls.reset()
        self.acc_overall_weighted.reset()

        return train_epoch_loss, jacc_miou_train, train_metrics_epoch

    def val_step(self, val_dataloader, epoch):
        running_val_loss = 0.0
        running_iou = []
        # Lists to store data for selected indices
        selected_indices_data = []
        print(f"val dataloder: {len(val_dataloader)}")

        # Eval Loop
        self.model.eval()
        for step, batch in enumerate(tqdm(val_dataloader)):
            batch_dict = {
                "pixel_values": batch["pixel_values"].to(self.device),
                "mask_labels": [labels.to(self.device)
                                for labels in batch["mask_labels"]],
                "class_labels": [labels.to(self.device)
                                 for labels in batch["class_labels"]],
                "pixel_mask": batch["pixel_mask"].to(self.device),
                "text_inputs": batch["text_inputs"].to(self.device),
                "task_inputs": batch["task_inputs"].to(self.device),
            }

            # Eval forward pass
            with torch.inference_mode():
                val_outputs = self.model(**batch_dict)

            val_loss = val_outputs.loss.item()
            # running_val_loss += val_loss
            batch_size = batch["pixel_values"].size(0)
            running_val_loss += val_loss * batch_size
            # Post process segmentation
            original_images = batch["original_images"]
            target_sizes = [(img.shape[-2], img.shape[-1])
                            for img in original_images]
            predicted_segmentation_maps = \
                self.processor.post_process_semantic_segmentation(
                    val_outputs, target_sizes=target_sizes)
            # Get ground truth segmentation maps & move target, preds to cpu
            target =  batch["original_segmentation_maps"].to(self.device)
            preds = torch.stack([
                t.detach() for t in predicted_segmentation_maps
            ]).to(self.device)

            # Metrics calculation per batch
            iou_per_cls_batch = self.val_iou_per_cls(preds.long(), target.long())
            acc_per_cls_batch = self.val_acc_per_cls(preds.long(), target.long())
            IoU_batch = self.val_IoU_weighted(preds.long(), target.long())
            acc_overall_batch = self.val_acc_overall_weighted(preds.long(), target.long())

            # Jackard metric - can be removed later it is same as IoU_weighted
            running_iou.append(self.val_jaccard_index(preds.long(), target.long()).item())

            print(f"Val Loss : {val_loss:.3f}\n")

            # Store one random index per step
            if not len(selected_indices_data) > 4:
                random_index = np.random.randint(0, batch_size)
                selected_indices_data.append(
                    (original_images[random_index],
                     target[random_index],
                     predicted_segmentation_maps[random_index])
                )
            # Log loss to wandb every 10 batches i.e after every 10 steps
            self.global_step += 1
            if ((step + 1) % 50) == 0 or step == len(val_dataloader) - 1:
                if self.log:
                    self.run.log({
                        "custom_step": int(self.global_step),
                        "epoch": epoch,
                        "Val_step_loss": val_loss
                    })
            del batch_dict, val_outputs

        running_val_loss = torch.tensor([running_val_loss], device=self.device)
        if self.is_distributed:
            torch.distributed.all_reduce(running_val_loss,
                                         op=torch.distributed.ReduceOp.SUM)
        val_epoch_loss = running_val_loss.item() / len(self.val_dataset)
        iou_per_cls_epoch = self.val_iou_per_cls.compute()
        mIoU_epoch = self.val_IoU_weighted.compute()
        jacc_miou_val = statistics.mean(running_iou)
        acc_per_cls_epoch = self.val_acc_per_cls.compute()
        acc_overall_epoch = self.val_acc_overall_weighted.compute()
        mean_acc_epoch = torch.nanmean(acc_per_cls_epoch)
        val_metrics_epoch = {
            "mean_iou": mIoU_epoch.item(),
            "mean_accuracy": mean_acc_epoch.item(),
            "overall_accuracy": acc_overall_epoch.item(),
            "per_category_iou": iou_per_cls_epoch,
            "per_category_accuracy": acc_per_cls_epoch
        }

        # Reset the metric states after the epoch
        self.val_iou_per_cls.reset()
        self.val_IoU_weighted.reset()
        self.val_acc_per_cls.reset()
        self.val_acc_overall_weighted.reset()

        # Wandb Logging of images and predictions at the end of validation epoch
        if self.log and self.device == 0:
            print(
                f"[INFO]: Logging Validation Images, on device: {self.device}"
            )
            print(len(selected_indices_data))
            table_data = log_selected_indices_data(selected_indices_data,
                                                   self.id2label)
            self.run.log({"predictions": table_data})
            del selected_indices_data
        return val_epoch_loss, jacc_miou_val, val_metrics_epoch

    def train(self):
        print("[INFO]: Started Training the model...")
        # Datasets
        train_dataset, val_dataset, train_queue, val_queue = self._get_datasets()
        self.val_dataset = val_queue
        # # DataLoaders
        # train_dataloader = self._prepare_dataloader(
        #     train_queue, batch_size=self.batch_size,
        #     is_train=self.is_train
        # )
        # print("train queue")
        # print(len(train_queue))
        # print(len(train_dataloader))

        print("Optimizer Parameter Groups:")
        for i, group in enumerate(self.optimizer.param_groups):
            print(f"Group {i} - Learning Rate: {group['lr']}")

        for epoch in tqdm(range(self.start_epoch, self.epochs)):
            print("epoch")
            val_dataloader = self._prepare_dataloader(
                val_queue, batch_size=self.batch_size, is_train=self.is_train
            )
            val_loss, jacc_miou_val, val_metrics_epoch = self.val_step(val_dataloader, epoch)
            # train_loss, jacc_miou_train, train_metrics_epoch = \
            # self.train_step(train_dataloader)
            # print("train_loss")
            # print(train_loss)
        # for step, batch in enumerate(train_dataloader):
        #     print(step)
        #     print(batch["original_images"].shape)
        #     print(batch["original_segmentation_maps"].shape)
            # print(batch.keys())
            # for k, v in batch.items():
            #     if isinstance(v, torch.Tensor):
            #         print(k, v.shape)
            #     else:
            #         print(k, len(v))


            # show(batch)
            # if step >= 20:
            #     break
        # return




In [41]:
import torch
import gc
import wandb
import subprocess, os, shutil

import argparse
import os
from typing import List, Dict, Tuple
import warnings
from datetime import datetime, timedelta


def main(run, args):
    if torch.cuda.is_available():
        rank = 0  # sets device=cuda:0 explicilty if gpu exist & only 1 available
    else:
        rank = "cpu"
    if args.is_train:
        print("[INFO]: Finetuning the Oneformer model with"
              f" {args.ckpt_model_name} backbone trained on"
              f" {args.ckpt_dataset_name} dataset on device {rank}.")
        trainer = OneformerTrainer(args, run, rank)
        model_results = trainer.train()
        return model_results

class myNamespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

args = myNamespace(is_train=True, ckpt_model_name="swin", ckpt_dataset_name="cityscapes",
                   num_epochs=2, lr=1e-4, batch_size=2, seed=1)
print(f"Cuda support: {torch.cuda.is_available()},"
        f"devices exist : {torch.cuda.device_count()}")
gc.collect()
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()

warnings.filterwarnings("ignore", category=FutureWarning)
wandb.login(key=os.environ.get("WANDB_API_KEY"))
# wandb_project_name = (
#         f"OF_Finetuning_{args.ckpt_dataset_name}_{args.ckpt_model_name}_"
#         f"railSem19_small_id2label"
#     )
wandb_project_name = "OF_Finetuning_cityscapes_swin_notebook"
wandb_config = dict(
        epochs=args.num_epochs,
        batch_size=args.batch_size,
        dataset=f"{args.ckpt_dataset_name}_RailSem19",
        architecture=f"OF_{args.ckpt_model_name}_L",
        project_name=wandb_project_name,
    )
print(f"[INFO]: Initializing wandb")
wandb.require("service")
run =  wandb.init(
        project=wandb_project_name,
        config=wandb_config,
        entity=os.environ.get("WANDB_ENTITY"),
        resume=True,
        mode="offline"
)
rdir = os.path.split(run.dir)[0]

model_results = main(run, args)

print("[INFO] Finishing the run....")
run.finish()
print("[INFO] Run Finished....")

if rdir:
    print(f"[INFO]: Syncing {rdir}")
    subprocess.run(["wandb", "sync", rdir])
    print(f"Removing {rdir}....")
    shutil.rmtree(rdir)

Cuda support: True,devices exist : 1
[INFO]: Initializing wandb


wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id tbb08rkz.


[INFO]: Finetuning the Oneformer model with swin backbone trained on cityscapes dataset on device 0.
[INFO]: Extracting Railsem19 dataset from hub on device 0
[INFO]: Found labels_info.json.Skipping Download...
[INFO]: id2label: {0: 'road', 1: 'sidewalk', 2: 'construction', 3: 'tram_track', 4: 'fence', 5: 'pole', 6: 'traffic_light', 7: 'traffic_sign', 8: 'vegetation', 9: 'terrain', 10: 'sky', 11: 'human', 12: 'rail_track', 13: 'car', 14: 'truck', 15: 'trackbed', 16: 'on_rails', 17: 'rail_raised', 18: 'rail_embedded', 19: 'background'}
[INFO]: Setting the seed 1
[INFO]: Prepare the Model and the Processor...
[INFO]: Model is in training mode...


Some weights of OneFormerForUniversalSegmentation were not initialized from the model checkpoint at shi-labs/oneformer_cityscapes_swin_large and are newly initialized: ['model.text_mapper.prompt_ctx.weight', 'model.text_mapper.text_encoder.ln_final.bias', 'model.text_mapper.text_encoder.ln_final.weight', 'model.text_mapper.text_encoder.positional_embedding', 'model.text_mapper.text_encoder.token_embedding.weight', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.bias', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm1.weight', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm2.bias', 'model.text_mapper.text_encoder.transformer.layers.0.layer_norm2.weight', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc1.bias', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc1.weight', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc2.bias', 'model.text_mapper.text_encoder.transformer.layers.0.mlp.fc2.weight', 'model.text_ma


-----Frozen Model Trainable Params......
trainable params: 35.33M || all params: 236.32M || trainable%: 14.95%
[INFO]: Num of params in prompt textmapper group: 81
[INFO]: Num of params in pretrained group: 801
[INFO]: Run object: <wandb.sdk.wandb_run.Run object at 0x7d39cb96ed40>
[INFO]: Started Training the model...
[INFO]: Preparing the Datasets...
[INFO]: No splits found.. Generating splits
[INFO]: Total images: 8500
[INFO]: Total Training images: 5
[INFO]: Total Validation images: 3
[INFO]: Found labels_info.json.Skipping Download...
class_mapping
{0: 'road', 1: 'sidewalk', 2: 'construction', 3: 'tram_track', 4: 'fence', 5: 'pole', 6: 'traffic_light', 7: 'traffic_sign', 8: 'vegetation', 9: 'terrain', 10: 'sky', 11: 'human', 12: 'rail_track', 13: 'car', 14: 'truck', 15: 'trackbed', 16: 'on_rails', 17: 'rail_raised', 18: 'rail_embedded', 19: 'background'}
class_coding
{'road': 0, 'sidewalk': 1, 'construction': 2, 'tram_track': 3, 'fence': 4, 'pole': 5, 'traffic_light': 6, 'traffic_

...

Patch List is empty.
Excepton: 'NoneType' object is not an iterator
dataloader datset

original_img shape <class 'PIL.JpegImagePlugin.JpegImageFile'>
original_img shape <class 'PIL.JpegImagePlugin.JpegImageFile'>
semantic mask shape <class 'PIL.PngImagePlugin.PngImageFile'>
semantic mask shape <class 'PIL.PngImagePlugin.PngImageFile'>
unique labels
unique labelstensor([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 12, 15, 16, 17, 19],
       dtype=torch.uint8)

tensor([ 0,  1,  2,  3,  4,  5,  7,  9, 10, 11, 12])tensor([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 19],
       dtype=torch.uint8)

transformed_image shape torch.Size([3, 1080, 1920])
dict_keys(['image', 'mask'])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
transformed_image shape torch.Size([3, 1080, 1920])
dict_keys(['image', 'mask'])
original_img shape <class 'PIL.JpegImagePlugin.JpegImageFile'>
semantic mask shape <class 'PIL.PngImagePlugin.PngImageFile'>
Extract patches
end
5
0 dict_keys(['imag

  0%|          | 0/8 [00:00<?, ?it/s]

my collate
collate
torch.Size([2, 3, 512, 512])
torch.Size([2, 512, 512])
Val Loss : 117.317

my collate
collate
torch.Size([2, 3, 512, 512])
torch.Size([2, 512, 512])
Val Loss : 112.351

Patch List is empty.
Extract patches
end
5
0 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
1 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
2 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
3 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
4 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
Extract patches
end
10
0 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
1 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
2 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])
3 dict_keys(['image', 'mask']) torch.Size([3, 512, 512]) torch.Size([1, 512, 512])

Val_step_loss,▁█
custom_step,▁█
epoch,▁█
Val_step_loss,122.18597
custom_step,16
epoch,1


[INFO] Run Finished....
[INFO]: Syncing /content/wandb/offline-run-20240227_091614-tbb08rkz
Removing /content/wandb/offline-run-20240227_091614-tbb08rkz....


We use the open-source CLIP (Radford et al.,
2021) with ViT-B/32 (Dosovitskiy et al., 2020) as
the teacher model. Its image encoder is a 12-layer
ViT with the hidden size to be 768 and 12 attention
heads. Its text encoder is a 12-layer Transformer
with hidden size to be 512 and 8 attention heads. (weughts tranferring)
-------------------------------
For the student model, we use ViT-S/16 with
hidden size to be 384 as the image encoder, and initialize it from the pre-trained weights on ImageNet21K (Ridnik et al., 2021).
For the text encoder,we experiment with 2, 4 and 6-layer Transformer,
of which the weights are initialized from the first corresponding layers of the teacher’s text encoder.

In [ ]:
## Imports
from typing import Tuple
import torch
from torch import nn
from transformers import CLIPTextModelWithProjection, AutoModelForUniversalSegmentation
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel, RobertaConfig, RobertaModel, RobertaEncoder
from torch.nn import CrossEntropyLoss, CosineEmbeddingLoss
import clip








In [ ]:
#create in oneformertrainer
def distill_oneformertextencoder(self.model):
    #get the text mapper
    student_model = self.model.model.text_mapper
    #use the model config to change the text_mapper configs
    self.model.config.text_encoder_num_layers = 2  # Change the number of layer to 2 instead of 6
    self.model.config.text_encoder_width = 512 #to match the clips embedding dimension , suhc that attention heads = 512 // 64 = 8 for each layer
    # Return the student model
    return student_model

## Distllator class
class TextDistillator(nn.Module):

    """
    A class to distillate a BERT-like model.
    """
    #self, teacher_model=None, student_model=None, temperature=None, lambda_param=None,  *args, **kwargs

    def __init__(self, teacher_model, student_model, temperature: float = 1.0, training=False) -> None:
        """
        Initiates the TextDistillator with the (teacher_model) to distillate from.
        """
        super(TextDistillator, self).__init__()
        self.student = student_model
        self.training = training
        self.temperature = temperature
        self.teacher = teacher_model
        # place teacher on same device as student
        self.teacher.to(self.student.device)
        self.teacher.eval()

    @property
    def temperature(self) -> float:
        return self._temperature if self.training else 1 #1 for inference

    @temperature.setter
    def temperature(self, value : float) -> None:
        if value < 1:
            raise(ValueError(f"Temperature must be above 1, it cannot be {value}"))
        self._temperature = value

    def get_logits(self, input_ids: Tensor=None, attention_mask: Tensor=None, from_teacher: bool = False) -> Tensor:
        """
        Given a couple of (input_ids) and (attention_mask), returns the logits corresponding to the prediction.
        The logits come from the student unless (from_teacher) is set to True, then it's from the teacher.
        """
        if from_teacher:
            return self.teacher.classifier(self.roberta(input_ids, attention_mask)[0])
        return self.student.classifier(self.student(input_ids, attention_mask)[0])

    def forward(self, input_ids : Tensor, attention_mask : Tensor, labels : Tensor) -> Tuple[Tensor, Tensor]:
        """
        Given a couple of (input_ids) and (attention_mask), returns the logits corresponding to the prediction.
        Also takes in the (labels) associated to the inputs.
        Returns the student probability distibution with temperature 1 and the loss.
        """
        student_logits = self.get_logits(input_ids, attention_mask, False)
        teacher_logits = self.get_logits(input_ids, attention_mask, True)
        return student_logits.softmax(1), self.loss(teacher_logits, student_logits, labels)

    def compute_loss(self, teacher_logits, student_logits, return_outputs=False):
        student_output = self.student(**inputs)

        with torch.no_grad():
          teacher_output = self.teacher(**inputs)

        # Compute soft targets for teacher and student
        soft_teacher = F.softmax(teacher_output.logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        # Compute the loss
        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        # Compute the true label loss
        student_target_loss = student_output.loss

        # Calculate final loss
        loss = (1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss
        return (loss, student_output) if return_outputs else loss

    # def loss(self,
    #     teacher_logits : Tensor,
    #     student_logits : Tensor,
    #     labels : Tensor,
    # ) -> Tensor:
    #     """
    #     The distillation loss for distilating a BERT-like model.
    #     The loss takes the (teacher_logits), (student_logits) and (labels) for various losses.
    #     """
    #     # Temperature and sotfmax
    #     student_logits, teacher_logits = (student_logits / self.temperature).softmax(1), (teacher_logits / self.temperature).softmax(1)
    #     # Classification loss (problem-specific loss)
    #     loss = CrossEntropyLoss()(student_logits, labels)
    #     # CrossEntropy teacher-student loss
    #     loss = loss + CrossEntropyLoss()(student_logits, teacher_logits)
    #     # Cosine loss
    #     loss = loss + CosineEmbeddingLoss()(teacher_logits, student_logits, torch.ones(teacher_logits.size()[0]))
    #     # Average the loss and return it
    #     loss = loss / 3
    #     return loss